In [1]:
import os
import sys
os.chdir('/Users/somanathnanda/baba/Covid-19/')

import pandas as pd
pd.options.display.max_columns=None
pd.set_option('display.max_rows', None)
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

import matplotlib
from plotly.offline import init_notebook_mode, iplot
import warnings

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True)

import tensorflow as tf
#tf.get_logger().setLevel('INFO')
#tf.autograph.set_verbosity(2)


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time
start = time.process_time()

In [2]:
def plot_timeseries(x_axis,data,name):
    num_cols=data.shape[1]
    go_arr=[]
    for i in range(num_cols):
        go_arr.append(go.Scatter(x=x_axis, y=data.iloc[:,i],name=data.columns[i]))   
    fig=go.Figure(go_arr)
    #fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 1, 0)',})
    fig.update_layout(
    title=name,font=dict(
        family="Courier New, monospace",
        color="#7f7f6a"))
    fig.show()
def plot_lines(df, title='', annotations=None):
    common_kw = dict(x=df.index, mode='lines+markers')
    xaxis = dict(title='Time Steps')
    data=[]
    for c in df.columns:
        data.append(go.Scatter(y=df[c], name=c, **common_kw))
    #data = [go.Scatter(y=df[c], name=c, **common_kw) for c in df.columns]
    layout = dict(title=title, showlegend=True, annotations=annotations, xaxis=xaxis)
    fig=dict(data=data, layout=layout)
    iplot(fig,show_link=True)

In [3]:
confirmed_data_df=pd.read_csv('data/time_series_covid19_confirmed_global.csv')
#recovered_data_df=pd.read_csv('data/time_series_covid19_recovered_global.csv')
death_data_df=pd.read_csv('data/time_series_covid19_deaths_global.csv')

country_pred_df=pd.DataFrame()
model_performance_df=pd.DataFrame()
forecast_df=pd.DataFrame()

def get_time_series_data(country_name):
    grouped_confirmed_df=confirmed_data_df.groupby('Country/Region')
    #grouped_recovered_df=recovered_data_df.groupby('Country/Region')
    grouped_death_df=death_data_df.groupby('Country/Region')
    country_confirmed_df=grouped_confirmed_df.get_group(country_name)
    #country_recovered_df=grouped_recovered_df.get_group(country_name)
    country_death_df=grouped_death_df.get_group(country_name)
    cleaned_confirmed_df=country_confirmed_df.drop(columns=['Province/State','Lat','Long']).sum(axis=0,skipna=True).to_frame()[1:]
    #cleaned_recovered_df=country_recovered_df.drop(columns=['Province/State','Lat','Long']).sum(axis=0,skipna=True).to_frame()[1:]
    cleaned_death_df=country_death_df.drop(columns=['Province/State','Lat','Long']).sum(axis=0,skipna=True).to_frame()[1:]
    ts_df=pd.DataFrame(cleaned_confirmed_df.values,columns=['confirmed_count'])
    #ts_df['recovered_count']=cleaned_recovered_df.values
    ts_df['death_count']=cleaned_death_df.values
    ts_df.index=cleaned_confirmed_df.index
    return ts_df
def plot(ts_df,country_name):
    #plot_timeseries(x_axis=ts_df.index,data=ts_df[['confirmed_count','recovered_count','death_count']],name='count_comparison_'+country_name)
    plot_lines(ts_df,'count_comparison_'+country_name)

# split a univariate sequence into samples
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
def build_model_and_predict(ts_df, country_name):
    tf.random.set_seed(1234)
    np.random.seed(1)
    time_steps = 5   
    rolling_days = 5
    n_test=2
    n_batch_size = 1
    X=series_to_supervised(ts_df[['confirmed_count']], n_in=time_steps, n_out=rolling_days, dropnan=True)
    train,test=X[:-n_test],X[-n_test:]
    X_train=train.iloc[:,:time_steps]
    Y_train = train.iloc[:,-rolling_days:]
    X_test=test.iloc[:,:time_steps]
    Y_test = test.iloc[:,-rolling_days:]
    
    X_train=X_train.values.astype('float32').reshape((X_train.shape[0], X_train.shape[1], 1))
    Y_train=Y_train.values.astype('float32').reshape((Y_train.shape[0], Y_train.shape[1], 1))
    X_test=X_test.values.astype('float32').reshape((X_test.shape[0], X_test.shape[1], 1))
    Y_test=Y_test.values.astype('float32').reshape((Y_test.shape[0], Y_test.shape[1], 1))
    
    c = [
        ModelCheckpoint(country_name+'.hdf5', save_best_only=True, monitor='val_loss', mode='min', verbose=2, save_freq=1),
        EarlyStopping(monitor='val_loss', min_delta=0, patience=500, verbose=2),
        ReduceLROnPlateau(monitor='val_loss',factor=0.2,patience=5,min_lr=0.00001)
    ]
    model = Sequential()
    model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=(time_steps,1)))
    model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(rolling_days))
    # compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mae','mse','accuracy'])
    # fit the model
    history=model.fit(X_train, Y_train, epochs=2000, batch_size=n_batch_size, verbose=2,callbacks=c,validation_data=(X_test, Y_test),shuffle=False)
    # evaluate the model
    [mse, mae, loss, accuracy]= model.evaluate(X_test, Y_test, batch_size=n_batch_size, verbose=1)
    model_performance=pd.DataFrame({'Country':[country_name],'mae':np.array(mae),'mse':np.array(mse),
                                    'rmse':np.array(np.sqrt(mse)),'accuracy':np.array(accuracy)})
    row = np.asarray(ts_df[-time_steps:].confirmed_count.values.astype('float32')).reshape((1, time_steps, 1))
    yhat=model.predict(row)
    return yhat,model_performance

In [4]:
country_names=['Ireland','India']#,'Sweden','Germany','Australia','Denmark','Iran','Austria','Canada','Singapore','Japan','Portugal','Brazil','US','Norway','United Kingdom']

In [ ]:
for country_name in country_names:
    ts_df=get_time_series_data(country_name)
    
    remove_the_number_of_zeros=(ts_df.confirmed_count==0).astype(int).sum()  #remove the number of zeroes from the begining. OIt can not appear in the middle
    ts_without_zeros_df=ts_df[remove_the_number_of_zeros-1:]
    
    yhat, model_performance = build_model_and_predict(ts_without_zeros_df,country_name)
    model_performance_df=model_performance_df.append(model_performance)
    final_date=pd.to_datetime(ts_df[-1:].index)
    pred_df=pd.DataFrame()
    for i in range(yhat.shape[1]):
        pred_df['Country']=[country_name]
        pred_df['pred_date']=final_date+(pd.DateOffset(i+1))
        pred_df['predicted_value']=round(yhat[0][i])
        forecast_df = forecast_df.append(pred_df)
        

Train on 22 samples, validate on 2 samples
Epoch 1/2000
22/22 - 3s - loss: 91502.3098 - mae: 201.1658 - mse: 91502.3203 - accuracy: 0.6818 - val_loss: 812163.7812 - val_mae: 736.5132 - val_mse: 812163.7500 - val_accuracy: 1.0000
Epoch 2/2000
22/22 - 0s - loss: 71054.1476 - mae: 168.6433 - mse: 71054.1484 - accuracy: 0.1364 - val_loss: 335966.0000 - val_mae: 508.7868 - val_mse: 335966.0000 - val_accuracy: 0.0000e+00
Epoch 3/2000
22/22 - 0s - loss: 78648.4701 - mae: 164.3607 - mse: 78648.4688 - accuracy: 0.7727 - val_loss: 752406.3438 - val_mae: 683.9174 - val_mse: 752406.3750 - val_accuracy: 1.0000
Epoch 4/2000
22/22 - 0s - loss: 184937.3749 - mae: 213.0447 - mse: 184937.3594 - accuracy: 0.6364 - val_loss: 827787.2500 - val_mae: 803.9750 - val_mse: 827787.2500 - val_accuracy: 0.0000e+00
Epoch 5/2000


22/22 - 0s - loss: 110486.2553 - mae: 213.5711 - mse: 110486.2500 - accuracy: 0.1818 - val_loss: 199235.7969 - val_mae: 403.4020 - val_mse: 199235.7969 - val_accuracy: 1.0000
Epoch 6/2000
22/22 - 0s - loss: 115088.1318 - mae: 187.4174 - mse: 115088.1328 - accuracy: 0.7273 - val_loss: 2902947.1875 - val_mae: 1622.1658 - val_mse: 2902947.2500 - val_accuracy: 1.0000
Epoch 7/2000
22/22 - 0s - loss: 373302.3729 - mae: 330.8879 - mse: 373302.3750 - accuracy: 0.5909 - val_loss: 1534190.8438 - val_mae: 938.9121 - val_mse: 1534190.8750 - val_accuracy: 1.0000
Epoch 8/2000
22/22 - 0s - loss: 295222.4273 - mae: 312.1878 - mse: 295222.4062 - accuracy: 0.1818 - val_loss: 1376192.6250 - val_mae: 836.8895 - val_mse: 1376192.6250 - val_accuracy: 0.0000e+00
Epoch 9/2000


22/22 - 0s - loss: 271517.6201 - mae: 253.7516 - mse: 271517.6250 - accuracy: 0.8182 - val_loss: 2433714.6875 - val_mae: 1338.7156 - val_mse: 2433714.7500 - val_accuracy: 1.0000
Epoch 10/2000
22/22 - 0s - loss: 161606.7934 - mae: 268.4862 - mse: 161606.7969 - accuracy: 0.8182 - val_loss: 385122.9375 - val_mae: 547.6505 - val_mse: 385122.9375 - val_accuracy: 1.0000
Epoch 11/2000
22/22 - 0s - loss: 96605.9024 - mae: 215.1019 - mse: 96605.8984 - accuracy: 0.7727 - val_loss: 179708.9922 - val_mae: 388.7158 - val_mse: 179709.0000 - val_accuracy: 1.0000
Epoch 12/2000
22/22 - 0s - loss: 80404.3261 - mae: 193.1165 - mse: 80404.3203 - accuracy: 0.8182 - val_loss: 774833.2656 - val_mae: 805.6794 - val_mse: 774833.2500 - val_accuracy: 1.0000
Epoch 13/2000


22/22 - 0s - loss: 78623.8988 - mae: 188.5404 - mse: 78623.9062 - accuracy: 0.9545 - val_loss: 1283406.6562 - val_mae: 1015.1685 - val_mse: 1283406.6250 - val_accuracy: 1.0000
Epoch 14/2000
22/22 - 0s - loss: 77661.1347 - mae: 189.6713 - mse: 77661.1406 - accuracy: 0.9545 - val_loss: 848003.0625 - val_mae: 799.5867 - val_mse: 848003.0625 - val_accuracy: 1.0000
Epoch 15/2000
22/22 - 0s - loss: 93150.3749 - mae: 197.3513 - mse: 93150.3750 - accuracy: 0.7727 - val_loss: 1176371.7188 - val_mae: 817.7705 - val_mse: 1176371.7500 - val_accuracy: 0.0000e+00
Epoch 16/2000
22/22 - 0s - loss: 66995.4289 - mae: 172.3326 - mse: 66995.4297 - accuracy: 0.8636 - val_loss: 386563.3984 - val_mae: 582.5838 - val_mse: 386563.4062 - val_accuracy: 1.0000
Epoch 17/2000
22/22 - 0s - loss: 61800.2656 - mae: 164.7520 - mse: 61800.2617 - accuracy: 0.9091 - val_loss: 355642.7656 - val_mae: 554.2042 - val_mse: 355642.7500 - val_accuracy: 1.0000
Epoch 18/2000


22/22 - 0s - loss: 60166.0482 - mae: 161.8571 - mse: 60166.0586 - accuracy: 0.9091 - val_loss: 333113.6719 - val_mae: 531.0592 - val_mse: 333113.6875 - val_accuracy: 1.0000
Epoch 19/2000
22/22 - 0s - loss: 58342.7508 - mae: 159.0082 - mse: 58342.7500 - accuracy: 0.9091 - val_loss: 320322.6641 - val_mae: 514.5365 - val_mse: 320322.6562 - val_accuracy: 1.0000
Epoch 20/2000
22/22 - 0s - loss: 56765.6110 - mae: 156.7291 - mse: 56765.6133 - accuracy: 0.9545 - val_loss: 388526.4141 - val_mae: 540.5602 - val_mse: 388526.4062 - val_accuracy: 1.0000
Epoch 21/2000
22/22 - 0s - loss: 55144.2975 - mae: 154.1251 - mse: 55144.2969 - accuracy: 0.9545 - val_loss: 467250.0156 - val_mae: 552.2359 - val_mse: 467250.0000 - val_accuracy: 0.5000
Epoch 22/2000


...

52/52 - 1s - loss: 666.8126 - mae: 12.5719 - mse: 666.8126 - accuracy: 0.5192 - val_loss: 38522.0625 - val_mae: 172.3717 - val_mse: 38522.0625 - val_accuracy: 1.0000
Epoch 259/2000
52/52 - 1s - loss: 666.6145 - mae: 12.5683 - mse: 666.6145 - accuracy: 0.5192 - val_loss: 38545.9688 - val_mae: 172.4058 - val_mse: 38545.9688 - val_accuracy: 1.0000
Epoch 260/2000


52/52 - 1s - loss: 666.1201 - mae: 12.5643 - mse: 666.1201 - accuracy: 0.5192 - val_loss: 38504.5059 - val_mae: 172.3517 - val_mse: 38504.5078 - val_accuracy: 1.0000
Epoch 261/2000
52/52 - 1s - loss: 665.8563 - mae: 12.5593 - mse: 665.8563 - accuracy: 0.5192 - val_loss: 38523.5352 - val_mae: 172.3872 - val_mse: 38523.5352 - val_accuracy: 1.0000
Epoch 262/2000


52/52 - 1s - loss: 665.4702 - mae: 12.5565 - mse: 665.4701 - accuracy: 0.5192 - val_loss: 38489.5156 - val_mae: 172.3261 - val_mse: 38489.5156 - val_accuracy: 1.0000
Epoch 263/2000
52/52 - 1s - loss: 665.2210 - mae: 12.5509 - mse: 665.2210 - accuracy: 0.5192 - val_loss: 38507.3984 - val_mae: 172.3632 - val_mse: 38507.3984 - val_accuracy: 1.0000
Epoch 264/2000


52/52 - 1s - loss: 664.8377 - mae: 12.5482 - mse: 664.8378 - accuracy: 0.5192 - val_loss: 38466.0195 - val_mae: 172.3021 - val_mse: 38466.0195 - val_accuracy: 1.0000
Epoch 265/2000
52/52 - 1s - loss: 664.6256 - mae: 12.5441 - mse: 664.6256 - accuracy: 0.5192 - val_loss: 38491.2188 - val_mae: 172.3363 - val_mse: 38491.2188 - val_accuracy: 1.0000
Epoch 266/2000


52/52 - 1s - loss: 664.0779 - mae: 12.5388 - mse: 664.0779 - accuracy: 0.5192 - val_loss: 38447.6699 - val_mae: 172.2737 - val_mse: 38447.6719 - val_accuracy: 1.0000
Epoch 267/2000
52/52 - 1s - loss: 663.9235 - mae: 12.5360 - mse: 663.9235 - accuracy: 0.5192 - val_loss: 38467.3730 - val_mae: 172.3111 - val_mse: 38467.3750 - val_accuracy: 1.0000
Epoch 268/2000


In [ ]:
for country_name in country_names:
    ts_df=get_time_series_data(country_name)
    df_1=ts_df[['confirmed_count']]
    df_1['date']=pd.to_datetime(df_1.index)
    df_1['date']=df_1['date'].dt.strftime('%Y-%m-%d')
    df_1.reset_index(drop=True, inplace=True)
    df_1.index=df_1['date']
    
    grouped_country_df=forecast_df.groupby('Country').get_group(country_name)
    grouped_country_df['pred_date']=grouped_country_df['pred_date'].dt.strftime('%Y-%m-%d')
    grouped_country_df.index=grouped_country_df.pred_date
    df_2=grouped_country_df['predicted_value']
    df_all=pd.merge(df_1,df_2,how='outer', left_index=True,right_index=True)
    
    plot(df_all,country_name)
    

In [ ]:
forecast_df

#### Compare the model accuracy for each country

In [ ]:
model_performance_df.index=model_performance_df['Country']
model_performance_df=model_performance_df.drop(columns=['Country'])
plot_lines(model_performance_df[['accuracy','mae','rmse']])

In [ ]:
print('Total Time taken : '+str((time.process_time() - start)/60)+' mins')